In [1]:
'''
    Store minimal dictionaries from full results for plotting
'''

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import pyreadr
import sys

sys.path.append('../utils_folder/')
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from matplotlib import rc as mtprc
mtprc('xtick', labelsize=20) 
mtprc('ytick', labelsize=20)

from utils_GD import *
from utils_IBP import *
from utils_plots import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
res = np.load('results/small_N_cancer_types.npy', allow_pickle=1).item()

_res = {}
_res[10] = {} #res[10]

for cancer in res[10].keys():
    _res[10][cancer] = {}
    num_its = len(res[10][cancer]['K'])
    _res[10][cancer]['N'] = res[10][cancer]['N']
    _res[10][cancer]['M'] = res[10][cancer]['M']
    _res[10][cancer]['K'] = res[10][cancer]['K']
    _res[10][cancer]['cts'] = res[10][cancer]['cts'][:,-1].reshape(num_its,1)
    _res[10][cancer]['IBP_preds'] = res[10][cancer]['IBP_preds'][:,-1].reshape(num_its,1)
    _res[10][cancer]['GD_preds'] = res[10][cancer]['GD_preds'][:,-1].reshape(num_its,1)
    _res[10][cancer]['J_preds'] = res[10][cancer]['J_preds'][:,:,-1].reshape(num_its,4,1)
    _res[10][cancer]['LP_preds'] = res[10][cancer]['LP_preds'][:,-1].reshape(num_its,1)
    _res[10][cancer]['GT_preds'] = res[10][cancer]['GT_preds'][:,0,0,-1].reshape(num_its,1,1,1)
    
np.save('results/_small_N_cancer_types.npy', _res)

In [22]:
results = np.load('results/folds_small_N_LOO_paper_cancer_types.npy',allow_pickle=1).item()
cancer_acc_ls = ['GBM', 'COADREAD', 'UCEC', 'LIHC', 'STAD']

_results = {}
_gbm = 0
_coad = 5
_ucec = 6
_lihc = 3
_stad = 7
_ls = [_gbm, _coad, _ucec, _lihc, _stad]

_results[10] = {}
for c, cancer in enumerate(cancer_acc_ls):
    _ = _ls[c]
    _results[10][cancer] = {}
    
    _results[10][cancer]['GD_mean'] = results[10][cancer]['GD_preds'][_].mean(axis = 0)
    _results[10][cancer]['GD_lo'] = np.quantile(results[10][cancer]['GD_preds'][_], q=.1, axis = 0)
    _results[10][cancer]['GD_hi'] = np.quantile(results[10][cancer]['GD_preds'][_], q=.9, axis = 0)
    
    _results[10][cancer]['IBP_mean'] = results[10][cancer]['IBP_preds'][_].mean(axis = 0)
    _results[10][cancer]['IBP_lo'] = np.quantile(results[10][cancer]['IBP_preds'][_], q=.1, axis = 0)
    _results[10][cancer]['IBP_hi'] = np.quantile(results[10][cancer]['IBP_preds'][_], q=.9, axis = 0)
    
    _results[10][cancer]['LP_mean'] = results[10][cancer]['LP_preds'][_].mean(axis = 0)
    _results[10][cancer]['LP_lo'] = np.quantile(results[10][cancer]['LP_preds'][_], q=.1, axis = 0)
    _results[10][cancer]['LP_hi'] = np.quantile(results[10][cancer]['LP_preds'][_], q=.9, axis = 0)
    
    _results[10][cancer]['J_mean'] = results[10][cancer]['J_preds'][_,:,-1].mean(axis = 0)
    _results[10][cancer]['J_lo'] = np.quantile(results[10][cancer]['J_preds'][_,:,-1], q=.1, axis = 0)
    _results[10][cancer]['J_hi'] = np.quantile(results[10][cancer]['J_preds'][_,:,-1], q=.9, axis = 0)
    
    _results[10][cancer]['GT_mean'] = results[10][cancer]['GT_preds'][_,:,0,0].mean(axis = 0)
    _results[10][cancer]['GT_lo'] = np.quantile(results[10][cancer]['GT_preds'][_,:,0,0], q=.1, axis = 0)
    _results[10][cancer]['GT_hi'] = np.quantile(results[10][cancer]['GT_preds'][_,:,0,0], q=.9, axis = 0)
    
    _results[10][cancer]['cts'] = results[10][cancer]['cts'][0]
    
np.save('results/_folds_small_N_LOO_paper_cancer_types.npy', _results)

In [30]:
res = np.load('results/small_N_paper_cancer_types_rare_variants.npy', allow_pickle=1).item()
_res = {}
_res[10] = {}
for c, cancer in enumerate(res[10]):
    print(cancer)
    _res[10][cancer] = {}
    GD_error = np.clip(np.abs(res[N][cancer_type]['GD_preds'][0, :, -1]-ctz)/ctz, 0, 1)
    GD_precision = 1-np.sort(GD_error)[lo_retain:hi_retain]
    _res[10][cancer]['GD_precision'] = GD_precision
    
    IBP_error = np.clip(np.abs(res[N][cancer_type]['IBP_preds'][0, :, -1]-ctz)/ctz, 0, 1)      
    IBP_precision = 1-np.sort(IBP_error)[lo_retain:hi_retain]
    _res[10][cancer]['IBP_precision'] = IBP_precision
    
    LP_error = np.clip(np.abs(res[N][cancer_type]['LP_preds'][0, :, -1]-ctz)/ctz, 0, 1)      
    LP_precision = 1-np.sort(LP_error)[lo_retain:hi_retain]
    _res[10][cancer]['LP_precision'] = LP_precision
    
np.save('results/_small_N_paper_cancer_types_rare_variants.npy', _res)

BRCA
LGG
LUAD
HNSC
COADREAD
LUSC
SKCM
UCEC
BLCA
OV


In [37]:
# res2 = np.load('results/paper_cancer_types.npy', allow_pickle=1).item()
# res = np.load('results/paper_cancer_types_rare_variants.npy', allow_pickle=1).item()
breast, ovarian, colorectal, sarcoma, glioblastoma = 'BRCA', 'OV', 'COADREAD', 'SARC', 'GBM'
selected_cancer_types = [breast, colorectal, ovarian, glioblastoma, sarcoma]
fraction = .05

_res = {}
_res[fraction] = {}
for c, cancer in enumerate(selected_cancer_types):
    print(cancer)
    ctz = res[fraction][cancer]['cts'][0,:,-1]
    
    _res[fraction][cancer] = {}
    _res[fraction][cancer]['N'] = res2[fraction][cancer]['N']
    _res[fraction][cancer]['cts'] = res[fraction][cancer]['cts'][0,:,-1]
    
    
    IBP_preds = res[fraction][cancer]['IBP_preds'][0,:,-1]
    GD_preds = res[fraction][cancer]['GD_preds'][0,:,-1]
    LP_preds = res[fraction][cancer]['LP_preds'][0,:,-1]

    IBP_EFPF = 1-np.clip(np.abs(IBP_preds-ctz)/ctz, 0, 1)
    GD_EFPF = 1-np.clip(np.abs(GD_preds-ctz)/ctz, 0, 1)
    LP = 1-np.clip(np.abs(LP_preds-ctz)/ctz, 0, 1)
    
    _res[fraction][cancer]['IBP_EFPF'] = IBP_EFPF
    _res[fraction][cancer]['GD_EFPF'] = GD_EFPF
    _res[fraction][cancer]['LP'] = LP


np.save('results/_paper_cancer_types_rare_variants.npy', _res)

BRCA
COADREAD
OV
GBM
SARC


In [46]:
error_dictionary = np.load('results/paper_calibration_error_dictionary.npy', allow_pickle = 1).item()
_error_dictionary = {}

GD_cov, IBP_cov = np.zeros([4, error_dictionary['percentage_checks'].shape[0], len(error_dictionary[.05].keys())]), np.zeros([4, error_dictionary['percentage_checks'].shape[0], len(error_dictionary[.05].keys())])
for f, fraction in enumerate([.05,.1,.2,.3]):
    for c, cancer in enumerate(error_dictionary[.05]):

        GD_cov[f,:,c] = error_dictionary[fraction][cancer]['GD_all'].mean(axis = 1)
        IBP_cov[f,:,c] = error_dictionary[fraction][cancer]['IBP_all'].mean(axis = 1)
_error_dictionary['GD_coverage'] = GD_cov
_error_dictionary['IBP_coverage'] = IBP_cov
_error_dictionary['percentage_checks'] = error_dictionary['percentage_checks']

np.save('results/_paper_calibration_error_dictionary.npy', _error_dictionary)